## LIBRERIAS

In [1]:
import pandas as pd
import plotly.graph_objects as go
import sys
import numpy as np
import plotly.express as px
import re
import stylecloud
import unicodedata
import nltk
import unicodedata
import pandas as pd
import warnings
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk import FreqDist
from PIL import Image
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",200)

## FUNCIONES

In [2]:
def clean_text(text, pattern="[^a-zA-Z0-9 ]"):
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, " ", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().strip().split())
    return cleaned_text
def completitud(df):
    comp=pd.DataFrame(df.isnull().sum())
    comp.reset_index(inplace=True)
    comp=comp.rename(columns={"index":"columna",0:"total"})
    comp["completitud"]=(1-comp["total"]/df.shape[0])*100
    comp=comp.sort_values(by="completitud",ascending=True)
    comp.reset_index(drop=True,inplace=True)
    return comp

def contar_vocales(texto):
    return len(re.findall("[aeiouáéíóíúü]", texto, re.IGNORECASE))


## PRACTICA

### <h1 style="color:green"> Parte 1 </h1>

#### DATASET

**Separador** "\n"

**Leemos el txt indicando el separador de lineas**

In [3]:
df=pd.read_csv("Chat de WhatsApp con Jaime.txt", sep="\n",encoding="latin1",error_bad_lines=False)

In [4]:
df.head()

,"01/12/19 9:29 a.Â m. - Los mensajes y las llamadas estÃ¡n cifrados de extremo a extremo. Nadie fuera de este chat, ni siquiera WhatsApp, puede leerlos ni escucharlos. Toca para obtener mÃ¡s informaciÃ³n."
0,"01/12/19 9:29 a.Â m. - Carla: Holi Jaime, te c..."
1,01/12/19 9:31 a.Â m. - Carla: Hoy estarÃ© fuer...
2,"01/12/19 9:57 a.Â m. - Jaime: Holi Carlita, vi..."
3,"01/12/19 9:58 a.Â m. - Jaime: Bambi Bambi , gr..."
4,01/12/19 4:00 p.Â m. - Jaime: Carlitaaa


**Renombramos la columna**

In [5]:
df.columns=["text"]

In [6]:
df.head()

,text
0,"01/12/19 9:29 a.Â m. - Carla: Holi Jaime, te c..."
1,01/12/19 9:31 a.Â m. - Carla: Hoy estarÃ© fuer...
2,"01/12/19 9:57 a.Â m. - Jaime: Holi Carlita, vi..."
3,"01/12/19 9:58 a.Â m. - Jaime: Bambi Bambi , gr..."
4,01/12/19 4:00 p.Â m. - Jaime: Carlitaaa


**A partir de la unica columna con la que contamos generamos dos columnas, la correspondiente a fecha y una más con el texto**
- Hacemos la separación del texto por ":"

In [7]:
df=df['text'].apply(lambda x: pd.Series(x.split(': '))).rename(columns={0:"d_fecha",1:"t_texto"})[["d_fecha","t_texto"]]

In [8]:
df["d_fecha"]

0       01/12/19 9:29 a.Â m. - Carla
1       01/12/19 9:31 a.Â m. - Carla
2       01/12/19 9:57 a.Â m. - Jaime
3       01/12/19 9:58 a.Â m. - Jaime
4       01/12/19 4:00 p.Â m. - Jaime
                    ...             
1128    29/10/20 9:33 p.Â m. - Jaime
1129    29/10/20 9:33 p.Â m. - Carla
1130    29/10/20 9:33 p.Â m. - Carla
1131    29/10/20 9:33 p.Â m. - Jaime
1132    29/10/20 9:33 p.Â m. - Jaime
Name: d_fecha, Length: 1133, dtype: object

**Extraemos el nombre de los usuarios de la variable "d_fecha"**

In [9]:
df["v_usuario"]=df["d_fecha"].map(lambda x:x.split("-")[1].strip() if len(x.split(" - "))==2 else np.nan)

**De la variable "d_date" extraemos la fecha, hora y formato, creamos un dataframe auxiliar**

In [10]:
df_aux=df["d_fecha"].apply(lambda x: pd.Series(x.split(' ')))[[0,1,2]].rename(columns={0:"d_fecha",1:"v_hora",2:"t_formato"})

In [11]:
df_aux.head()

,d_fecha,v_hora,t_formato
0,01/12/19,9:29,a.Â m.
1,01/12/19,9:31,a.Â m.
2,01/12/19,9:57,a.Â m.
3,01/12/19,9:58,a.Â m.
4,01/12/19,4:00,p.Â m.


**Agregamos la información al dataframe original**

In [12]:
df=pd.concat([df[["t_texto","v_usuario"]],df_aux],axis=1)

**Extraemos de la variable "v_hora" la hora , omitimos los minutos**

In [13]:
df["v_hora"].map(lambda x:str(x).split(":")[0]).value_counts(1)

10             0.131509
9              0.122683
7              0.122683
1              0.086496
12             0.073257
11             0.070609
4              0.063548
8              0.062665
2              0.054722
6              0.045896
3              0.045013
               0.030009
5              0.028244
nan            0.011474
de             0.008826
mediante       0.005296
(EE.           0.003530
MÃ©xico        0.003530
India)         0.003530
(Singapur)     0.001765
(CanadÃ¡)      0.001765
su             0.001765
a              0.001765
(Mumbay        0.001765
(Alemania)     0.001765
(Brasil)       0.001765
(JapÃ³n)       0.001765
favor,         0.001765
segÃºn         0.001765
PaÃ­ses        0.001765
(Hyderabad     0.001765
con            0.001765
(Australia)    0.001765
(Amsterdam     0.001765
Name: v_hora, dtype: float64

**Agregamos una condicion para filtrar solo los registros que son de tipo digito y los que no cumplan la condición los remplazamos por NaN**

In [14]:
df["v_hora"].map(lambda x:str(x).split(":")[0] if str(x).split(":")[0].strip().isdigit() else np.nan)

0       9
1       9
2       9
3       9
4       4
       ..
1128    9
1129    9
1130    9
1131    9
1132    9
Name: v_hora, Length: 1133, dtype: object

In [15]:
df["v_hora"]=df["v_hora"].map(lambda x:str(x).split(":")[0] if str(x).split(":")[0].strip().isdigit() else np.nan)

In [16]:
df["v_hora"].value_counts()

10    149
7     139
9     139
1      98
12     83
11     80
4      72
8      71
2      62
6      52
3      51
5      32
Name: v_hora, dtype: int64

**Lo convertimos a formato de 24 horas** 

In [17]:
df["t_formato"]=df["t_formato"].map(lambda x:str(x).strip())

In [18]:
df["t_formato"].value_counts(1)

p.Â m.       0.631951
a.Â m.       0.275375
nan          0.034422
             0.030009
reuniÃ³n     0.005296
acceso       0.003530
UU.          0.003530
Bajos)       0.001765
su           0.001765
H.323        0.001765
descargue    0.001765
SIP          0.001765
nÃºmero      0.001765
la           0.001765
un           0.001765
Skype        0.001765
Name: t_formato, dtype: float64

In [19]:
list(df["t_formato"].value_counts().index[:2])

['p.Â\xa0m.', 'a.Â\xa0m.']

In [20]:
am_pm=list(df["t_formato"].value_counts().index[:2])

**Seleccionamos solo los registros que tienen "am" y "pm" en "t_formato"**

In [21]:
df.shape

(1133, 5)

In [22]:
df[df["t_formato"].isin(am_pm)].reset_index(drop=True)

,t_texto,v_usuario,d_fecha,v_hora,t_formato
0,"Holi Jaime, te compartÃ­ una carpeta en drive ...",Carla,01/12/19,9,a.Â m.
1,Hoy estarÃ© fuera d emi Casita pero si necesit...,Carla,01/12/19,9,a.Â m.
2,"Holi Carlita, vientos, muchas gracias, eres la...",Jaime,01/12/19,9,a.Â m.
3,"Bambi Bambi , graciaaas",Jaime,01/12/19,9,a.Â m.
4,Carlitaaa,Jaime,01/12/19,4,p.Â m.
...,...,...,...,...,...
1023,El de el creo si lo hizo chido,Jaime,29/10/20,9,p.Â m.
1024,Va va va ð¬,Carla,29/10/20,9,p.Â m.
1025,Se lo voa pedir âºï¸âºï¸,Carla,29/10/20,9,p.Â m.
1026,Si si si,Jaime,29/10/20,9,p.Â m.


In [23]:
df=df[df["t_formato"].isin(am_pm)].reset_index(drop=True)

**Añadimos 12 horas a los que son pm y los de am se mantienen igual**

In [24]:
df["v_hora"]=df[["v_hora","t_formato"]].apply(lambda x:int(x[0])+12 if "p" in x[1] else x[0],axis=1)

In [25]:
df.head()

,t_texto,v_usuario,d_fecha,v_hora,t_formato
0,"Holi Jaime, te compartÃ­ una carpeta en drive ...",Carla,01/12/19,9,a.Â m.
1,Hoy estarÃ© fuera d emi Casita pero si necesit...,Carla,01/12/19,9,a.Â m.
2,"Holi Carlita, vientos, muchas gracias, eres la...",Jaime,01/12/19,9,a.Â m.
3,"Bambi Bambi , graciaaas",Jaime,01/12/19,9,a.Â m.
4,Carlitaaa,Jaime,01/12/19,16,p.Â m.


<h1 style="color:blue;" > 3.a)</h1>

In [26]:
missings=completitud(df)

In [27]:
missings

,columna,total,completitud
0,t_texto,0,100.0
1,v_usuario,0,100.0
2,d_fecha,0,100.0
3,v_hora,0,100.0
4,t_formato,0,100.0


<h1 style="color:blue;" > 3.b)</h1>

**Convertimos en minusculas el texto para buscar los registros que tienen "multimedia omitido" dentro**

In [28]:
df["t_texto"]=df["t_texto"].map(lambda x:str(x).lower())

In [29]:

df[df["t_texto"].map(lambda x:"omitido" in x)]["t_texto"].value_counts()

<multimedia omitido>    84
Name: t_texto, dtype: int64

In [30]:
total_multimedia=df[df["t_texto"].map(lambda x:"omitido" in x)].shape[0]

In [31]:
total_multimedia

84

**Conservamos los registros que no contienen "multimedia omitido"**

In [32]:

df[~df["t_texto"].map(lambda x:"omitido" in x)]

,t_texto,v_usuario,d_fecha,v_hora,t_formato
0,"holi jaime, te compartã­ una carpeta en drive ...",Carla,01/12/19,9,a.Â m.
1,hoy estarã© fuera d emi casita pero si necesit...,Carla,01/12/19,9,a.Â m.
2,"holi carlita, vientos, muchas gracias, eres la...",Jaime,01/12/19,9,a.Â m.
3,"bambi bambi , graciaaas",Jaime,01/12/19,9,a.Â m.
4,carlitaaa,Jaime,01/12/19,16,p.Â m.
...,...,...,...,...,...
1023,el de el creo si lo hizo chido,Jaime,29/10/20,21,p.Â m.
1024,va va va ð¬,Carla,29/10/20,21,p.Â m.
1025,se lo voa pedir âºï¸âºï¸,Carla,29/10/20,21,p.Â m.
1026,si si si,Jaime,29/10/20,21,p.Â m.


In [33]:
df=df[~df["t_texto"].map(lambda x:"omitido" in x)].reset_index(drop=True)

In [34]:
#corroboramos que no tengamos más registros
df[df["t_texto"].map(lambda x:"omitido" in x)]

,t_texto,v_usuario,d_fecha,v_hora,t_formato


<h1 style="color:blue;" > 3.c)</h1>

In [35]:
for i in df:
    print(i)
    display(df[i].value_counts())

t_texto


jajaja                                                                      28
va va va                                                                    17
xd                                                                          14
jajajaj                                                                     12
ð±                                                                        12
                                                                            ..
si si si                                                                     1
right now ð¤£                                                               1
me dijo la nutriã³loga que lo puedes cambiar por salmon si quieres jajaj     1
la liga de la clase!!!!                                                      1
ð¯                                                                         1
Name: t_texto, Length: 753, dtype: int64

v_usuario


Jaime    522
Carla    422
Name: v_usuario, dtype: int64

d_fecha


13/10/20    138
15/10/20     94
28/10/20     56
29/10/20     37
17/10/20     31
           ... 
09/09/20      2
08/10/20      2
03/08/20      2
21/09/20      1
02/12/19      1
Name: d_fecha, Length: 66, dtype: int64

v_hora


19    127
10    101
21     75
13     71
16     65
9      59
11     56
20     55
14     53
15     48
18     46
24     41
22     36
12     32
17     25
23     17
1      17
8      15
7       3
2       2
Name: v_hora, dtype: int64

t_formato


p.Â m.    659
a.Â m.    285
Name: t_formato, dtype: int64

**En las variables categóricas así como en la variable de fecha no existen inconsistencias**  

<h1 style="color:blue;" > 3.d)</h1>

In [36]:
df["v_usuario"].value_counts()

Jaime    522
Carla    422
Name: v_usuario, dtype: int64

In [37]:
df["v_usuario"]=df["v_usuario"].map(clean_text)

In [38]:
df["v_usuario"].value_counts()

jaime    522
carla    422
Name: v_usuario, dtype: int64

<h1 style="color:blue;" > 3.e)</h1>

**En este caso solo contamos con dos categorias, y los nombres no contienen carácteres especiales, solo las convertiremos a minusculas**

In [39]:
df["d_fecha"]=pd.to_datetime(df["d_fecha"])

<h1 style="color:blue;" > 3.f)</h1>

In [40]:
df["v_hora"]=df["v_hora"].astype(int)

<h1 style="color:blue;" > 3.g)</h1>

In [41]:
df["t_texto_ori"]=df["t_texto"]
df["t_texto"]=df["t_texto"].map(lambda x:clean_text(x))


**Verificamos que no existan registros sin información, si es así deben eliminarse de la tabla**

In [42]:
df[df["t_texto"]==""]

,t_texto,v_usuario,d_fecha,v_hora,t_formato,t_texto_ori
26,,jaime,2019-06-12,19,p.Â m.,ð
66,,carla,2020-04-14,13,p.Â m.,ð§¡
81,,jaime,2020-04-24,18,p.Â m.,ð
86,,jaime,2020-04-24,20,p.Â m.,ððð¼ðð¼
164,,carla,2020-07-30,17,p.Â m.,?
...,...,...,...,...,...,...
849,,jaime,2020-10-27,20,p.Â m.,ð¥º
857,,jaime,2020-10-28,10,a.Â m.,ð¥³ð¥³ð¥³
890,,carla,2020-10-28,24,p.Â m.,ð¬
908,,jaime,2020-10-29,14,p.Â m.,ð±


**Existen 61 registros sin información después de la limpieza, así que los removemos**

In [43]:
df=df[df["t_texto"]!=""]
df.reset_index(drop=True,inplace=True)

<h1 style="color:blue;" > 3.h)</h1>

**Añadimos algunas stop words adicionales para una mejor limieza del texto**

In [44]:
lista_stopwords = stopwords.words("spanish")
lista_stopwords=lista_stopwords+["xd","va","ja","jaja","jajaja","si","ok"]

In [45]:
df["t_texto"]=df["t_texto"].map(lambda x:" ".join(list(filter(lambda x:x not in lista_stopwords,x.split()))).strip())

**Verificamos que no existan registros sin información, si es así deben eliminarse de la tabla**

In [46]:
df[df["t_texto"]==""]

,t_texto,v_usuario,d_fecha,v_hora,t_formato,t_texto_ori
29,,carla,2019-06-12,19,p.Â m.,jajaja
43,,jaime,2020-02-01,11,a.Â m.,jajaja
53,,jaime,2020-01-23,22,p.Â m.,jajaja
67,,carla,2020-04-14,13,p.Â m.,va va va
73,,jaime,2020-04-14,13,p.Â m.,jajaja
...,...,...,...,...,...,...
870,,carla,2020-10-29,21,p.Â m.,jajaja ð¤£ ok ok
872,,carla,2020-10-29,21,p.Â m.,ya estã¡s
874,,jaime,2020-10-29,21,p.Â m.,jajaja
879,,carla,2020-10-29,21,p.Â m.,va va va ð¬


In [47]:
df=df[df["t_texto"]!=""]
df.reset_index(drop=True,inplace=True)

**Eliminamos aquellos registros donde aún contamos con "jaja" y además la longitud de la palabra es inferior a seis, además seleccionamos solo los registros con longitud superior a dos**

In [48]:
df=df[~df["t_texto"].map(lambda x:"jaja" in x and len(x)<6)].reset_index(drop=True)
df=df[df["t_texto"].map(lambda x:len(x)>2)].reset_index(drop=True)

In [49]:
df[df["t_texto"]==""]

,t_texto,v_usuario,d_fecha,v_hora,t_formato,t_texto_ori


<h1 style="color:blue;" > 3.i)</h1>

In [50]:
#Obtenemos el corpus del texto
corpus=" ".join(df["t_texto"].values)
#Obtenemos la frecuencia de la spalabras con FreqDist
fdist=nltk.FreqDist(corpus.split())
#Seleccionamos los hapaxes
list_hapaxes=fdist.hapaxes()

**Eliminamos los hapaxes**

In [51]:
df["t_texto"]=df["t_texto"].map(lambda text:" ".join([x for x in text.split(" ") if x not in list_hapaxes]))
df["t_texto"]

0      holi jaime comparta carpeta estan rangos csv a...
1                    hoy d casita necesitas llegue checo
2                      holi carlita muchas gracias mejor
3                                            bambi bambi
4                                              carlitaaa
                             ...                        
778                                       puedo pedir 10
779                                                mm sr
780                                      creo hizo chido
781                                       voa pedir aiai
782                                                mejor
Name: t_texto, Length: 783, dtype: object

In [52]:
df[df["t_texto"] == ""]

,t_texto,v_usuario,d_fecha,v_hora,t_formato,t_texto_ori
20,,jaime,2019-01-12,22,p.Â m.,graciaas
29,,jaime,2019-06-12,19,p.Â m.,aaa ok ok
51,,jaime,2020-01-23,22,p.Â m.,ya te lo autorize
60,,carla,2020-04-14,13,p.Â m.,"si si , me agrada la idea"
91,,jaime,2020-05-29,16,p.Â m.,no tienes banca movil?
...,...,...,...,...,...,...
746,,carla,2020-10-28,20,p.Â m.,amossss
749,,jaime,2020-10-28,20,p.Â m.,vaaaaa
759,,jaime,2020-10-29,14,p.Â m.,perdã³on
764,,jaime,2020-10-29,14,p.Â m.,y eso que vine temprano


**Eliminamos registros vacios**

In [53]:
df = df[df["t_texto"] != ""]
df.reset_index(drop=True,inplace=True)
df.head()

,t_texto,v_usuario,d_fecha,v_hora,t_formato,t_texto_ori
0,holi jaime comparta carpeta estan rangos csv a...,carla,2019-01-12,9,a.Â m.,"holi jaime, te compartã­ una carpeta en drive ..."
1,hoy d casita necesitas llegue checo,carla,2019-01-12,9,a.Â m.,hoy estarã© fuera d emi casita pero si necesit...
2,holi carlita muchas gracias mejor,jaime,2019-01-12,9,a.Â m.,"holi carlita, vientos, muchas gracias, eres la..."
3,bambi bambi,jaime,2019-01-12,9,a.Â m.,"bambi bambi , graciaaas"
4,carlitaaa,jaime,2019-01-12,16,p.Â m.,carlitaaa


In [54]:
df[df["t_texto"]==""]


,t_texto,v_usuario,d_fecha,v_hora,t_formato,t_texto_ori


In [55]:
df.shape

(690, 6)

**Volvemos a filtrar aquellos registros donde la longitud sea inferior o igual a dos letras**

In [56]:
df[~df["t_texto"].map(lambda x:len(x)>2)].reset_index(drop=True)

,t_texto,v_usuario,d_fecha,v_hora,t_formato,t_texto_ori
0,mm,jaime,2019-01-12,19,p.Â m.,mm como cuantos salen decentemente?
1,5,jaime,2020-05-29,17,p.Â m.,012 180 01536067603 5
2,di,jaime,2020-07-10,9,a.Â m.,no te di chance de prepararte xd
3,ma,jaime,2020-10-15,19,p.Â m.,no ma soy amigo de la profa
4,10,carla,2020-10-23,18,p.Â m.,sacas 10


In [57]:
df=df[df["t_texto"].map(lambda x:len(x)>2)].reset_index(drop=True)

In [58]:
df[~df["t_texto"].map(lambda x:len(x)>2)].reset_index(drop=True)

,t_texto,v_usuario,d_fecha,v_hora,t_formato,t_texto_ori


In [59]:
#Creamos una variable del texto limpio
df["t_texto_clean"] = df["t_texto"]

<h1 style="color:blue;" > 3.j)</h1>

**Generamos tokens del texto limpio**

In [60]:
df["t_texto"]=df["t_texto"].map(lambda x:x.split())
df["t_texto"].head()

0    [holi, jaime, comparta, carpeta, estan, rangos...
1           [hoy, d, casita, necesitas, llegue, checo]
2              [holi, carlita, muchas, gracias, mejor]
3                                       [bambi, bambi]
4                                          [carlitaaa]
Name: t_texto, dtype: object

<h1 style="color:blue;" > 3.k)</h1>

In [61]:
from nltk.stem import SnowballStemmer
spanish_stemmer = SnowballStemmer('spanish')

df["t_texto"]=df["t_texto"].map(lambda text:[spanish_stemmer.stem(word) for word in text])
df["t_texto"].head()

0    [holi, jaim, compart, carpet, estan, rang, csv...
1                 [hoy, d, casit, necesit, lleg, chec]
2                   [holi, carlit, much, graci, mejor]
3                                       [bambi, bambi]
4                                           [carlitaa]
Name: t_texto, dtype: object

<h1 style="color:blue;" > 3.l)</h1>

<h1 style="color:orange;" > a) Análisis de longitud</h1>

In [62]:
#Número de palabras
df['c_numero_palabras'] = df["t_texto"].apply(lambda x: len(x))

#Número de letras
df['c_numero_letras'] =df["t_texto"].map(lambda x: sum(len(word) for word in x))

#Longitud promedio de palabras
df['c_lon_pro_pal'] = df['c_numero_letras'] / df['c_numero_palabras']
#Número de vocales en el texto limpio
df["c_num_vocales"]=df["t_texto_clean"].map(contar_vocales)
df.head()

,t_texto,v_usuario,d_fecha,v_hora,t_formato,t_texto_ori,t_texto_clean,c_numero_palabras,c_numero_letras,c_lon_pro_pal,c_num_vocales
0,"[holi, jaim, compart, carpet, estan, rang, csv...",carla,2019-01-12,9,a.Â m.,"holi jaime, te compartã­ una carpeta en drive ...",holi jaime comparta carpeta estan rangos csv a...,10,51,5.1,25
1,"[hoy, d, casit, necesit, lleg, chec]",carla,2019-01-12,9,a.Â m.,hoy estarã© fuera d emi casita pero si necesit...,hoy d casita necesitas llegue checo,6,24,4.0,13
2,"[holi, carlit, much, graci, mejor]",jaime,2019-01-12,9,a.Â m.,"holi carlita, vientos, muchas gracias, eres la...",holi carlita muchas gracias mejor,5,24,4.8,12
3,"[bambi, bambi]",jaime,2019-01-12,9,a.Â m.,"bambi bambi , graciaaas",bambi bambi,2,10,5.0,4
4,[carlitaa],jaime,2019-01-12,16,p.Â m.,carlitaaa,carlitaaa,1,8,8.0,5


<h1 style="color:orange;" > b) Análisis de sentimientos</h1>

pip install translate


https://www.tutorialspoint.com/python_text_processing/python_text_translation.htm

In [63]:
from translate import Translator
translator= Translator(from_lang="spanish",to_lang="english")

def eng_to_spa(text):
    try:
        aux=translator.translate(text)
    except:
        aux=""
    return aux

In [ ]:
df["t_texto_eng"]=df["t_texto_clean"].map(eng_to_spa)

In [ ]:
df[df["t_texto_eng"]!=""].shape

In [ ]:
df[df["t_texto_eng"]==""].shape

In [ ]:
df["c_polarity"]=df["t_texto_eng"].map(lambda x: TextBlob(x).sentiment.polarity)

In [ ]:
df

<h1 style="color:orange;" > c) Reconocimiento de entidad</h1>

In [ ]:
import spacy
ner = spacy.load('en_core_web_sm')
from collections import Counter

In [ ]:
df["docs"]=df["t_texto_eng"].map(lambda txt:ner(txt))

df["1"]=df["docs"].map(lambda x:Counter([y.label_ for y in x.ents]))
#Counter nos ayudará a generar un diccionario, donde los valores son las precuencias con la que 
#aparece la palabra

df['1'].apply(pd.Series).fillna(0)
#utilizamos apply y series para generar columnas a partir de los valores de los diccionarios

df=pd.concat([df,df['1'].apply(pd.Series).fillna(0)],axis=1)
#añadimos las columnas creadas a la tabla que contiene el resto de la información

In [ ]:
df.drop(columns=["docs","1"],inplace=True)

In [ ]:
df=df.rename(columns={"PERSON":"c_person","DATE":"c_date","NORP":"c_norp","ORG":"c_org","TIME":"c_time","CARDINAL":"c_cardinal"})

In [ ]:
df.head()

<h1 style="color:orange;" > c) TFIDFVectorizer</h1>

In [ ]:
#Generamos en conjunto de entrenamiento y el de prueba
from sklearn.model_selection import train_test_split
X_train,X_test=train_test_split(df, test_size=0.25, random_state=413)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_df=.3,min_df=0.01)#Seleccionamos solo aquellas palabras que aparecen un mínimo del 1%
# y algunas más con una presencia de a lo más un 30%
vect.fit(X_train["t_texto_clean"])
array_train = vect.transform(X_train["t_texto_clean"])#Se transforman los datos de train para obtener el conteo por palabra
array_test = vect.transform(X_test["t_texto_clean"])#Se aplica lo mismo para test

**También se puede hacer con el texto derivado pero algunas de las palabras no son interpretables**


In [ ]:
#vect = TfidfVectorizer(max_df=.3,min_df=0.01)
#vect.fit(X_train["t_texto"].map(lambda x:" ".join(x)))
#array_train = vect.transform(X_train["t_texto"].map(lambda x:" ".join(x)))#Se transforman los datos de train para obtener el conteo por palabra
#array_test = vect.transform(X_test["t_texto"].map(lambda x:" ".join(x)))#Se aplica lo mismo para test

In [ ]:
aux_train=pd.DataFrame(array_train.toarray(),columns=vect.get_feature_names())
aux_test=pd.DataFrame(array_test.toarray(),columns=vect.get_feature_names())

In [ ]:
aux_test.shape

In [ ]:
aux_train.shape

In [ ]:
#Añadimos las nuevas columnas al dataset train 
X_train=pd.concat([X_train.reset_index(drop=True),aux_train],axis=1)
#Agregamos el reset index ya que concat e basa en los indices para añadir nueva información 
#Añadimos las nuevas columnas al dataset test 
X_test=pd.concat([X_test.reset_index(drop=True),aux_test],axis=1)
X_train.head()

**Verificamos que el cruce no nos genere nulos**

In [ ]:
X_train.isnull().sum()

In [ ]:
X_test.isnull().sum()

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
X_test[X_test["t_texto_clean"]==""]

In [ ]:
X_train[X_train["t_texto_clean"]==""]

### <h1 style="color:green"> Parte 2 </h1>

In [ ]:
df = pd.read_csv("BOLSAA.MX.csv")

<h1 style="color:blue;" > a)</h1>

In [ ]:
c_feats = ['Open','High','Low','Close','Adj Close','Volume']
c_feats_new= ["c_" + x for x in c_feats]
df.rename(columns = dict(zip(c_feats,c_feats_new)), inplace=True)
df.rename(columns = {'date':'d_Date'}, inplace=True)

In [ ]:
df.head()

<h1 style="color:blue;" > b)</h1>

**Verificamos duplicados**

In [ ]:
df.duplicated().sum()

In [ ]:
df.duplicated(subset=["d_Date"]).sum()

**Verificamos completitud**

In [ ]:
completitud(df)

**Verificamos tipo de dato**

In [ ]:
df.dtypes

In [ ]:
df["d_Date"]=pd.to_datetime(df["d_Date"])

In [ ]:
df.describe(percentiles=np.arange(0.1,1.1,.1))

**Eliminamos aquellos registros donde no se tuvo movimiento, es decir , cuando el volumen de las acciones es cero**


In [ ]:
df=df[df["c_Volume"]>0].reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
df

**Verificamos que la información no sea de fechas que aún no suceden**

In [ ]:
df["d_Date"].min()

In [ ]:
df["d_Date"].max()

<h1 style="color:blue;" > c)</h1>

In [ ]:
df=df.sort_values(by=["d_Date"])
df.reset_index(drop=True,inplace=True)
df.head()

<h1 style="color:blue;" > d)</h1>

**Los registros se toman una vez al día**

In [ ]:
df["d_Date"].diff().mean()

<h1 style="color:blue;" > e)</h1>

In [ ]:
df_ori=df.copy()

In [ ]:
df_ori.rename(columns={"c_Close":"tgt"},inplace=True)

In [ ]:
#Realizamos el desfase de tal forma que las características permanezcan igual , sin embargo el valor a cierre 
#corresponde a dos días posteriores, es decir, con la información de hot intentó predecir el valor de cierre que tendrá
#el dos días

df_ori["tgt"].shift(-2)

In [ ]:
df_ori["tgt"]=df_ori["tgt"].shift(-2)

In [ ]:
df_ori["c_diff_max_min"]=df_ori["c_High"]-df_ori["c_Low"]

**Las variables pct_cie_ape y ratio no se pueden realizar ya que necesitamos conocer el precio de cierre
que tendrá dentro de dos días para poder añadir esta variable, la única forma de añadir esta información
es si se realiza un desfase de tgt pero de días anteriores , de esa forma son valores que conocemos y podemos 
realizar ingeniería sobre ellos, por otro lado estas variables están altamente correlacionadas con la tgt
por esa razón no se construyen**

In [ ]:
for col in ['c_Open', 'c_High', 'c_Low', 'c_Volume']:
    for i in range(3):
        df_ori[f"{col}_diff_{i+1}"] = df_ori[f"{col}"].diff(i+1)
        df_ori[f"{col}_win_{i+1}"] = df_ori[f"{col}"].rolling(i+2).mean()
        df_ori[f"{col}_pct_{i+1}"] = df_ori[f"{col}"].pct_change(i+1)
        df_ori[f"{col}_2"] = df_ori[f"{col}"].map(lambda x: x*x)

In [ ]:
df_ori.head()

**Eliminados los NaN**

In [ ]:
df_ori=df_ori.dropna().reset_index(drop=True)

<h1 style="color:blue;" > f)</h1>

In [ ]:
def minPctChange(x):
    return x.pct_change().replace([np.inf, -np.inf], np.nan).min()
def maxPctChange(x):
    return x.pct_change().replace([np.inf, -np.inf], np.nan).max()
def meanPctChange(x):
    return x.pct_change().replace([np.inf, -np.inf], np.nan).mean()
def stdPctChange(x):
    return x.pct_change().replace([np.inf, -np.inf], np.nan).std()
def sumPctChange(x):
    return x.pct_change().replace([np.inf, -np.inf], np.nan).sum()
def minChange(x):
    return x.diff().replace([np.inf, -np.inf], np.nan).min()
def maxChange(x):
    return x.diff().replace([np.inf, -np.inf], np.nan).max()
def meanChange(x):
    return x.diff().replace([np.inf, -np.inf], np.nan).mean()
def stdChange(x):
    return x.diff().replace([np.inf, -np.inf], np.nan).std()
def sumChange(x):
    return x.diff().replace([np.inf, -np.inf], np.nan).sum()

In [ ]:
df_3_days=df.copy()

In [ ]:

#Separamos características de la target
X=df_3_days[[x for x in df_3_days if x!="c_Close"]]
y=df_3_days[["c_Close","d_Date"]].rename(columns={"c_Close":"tgt"})

**Realizamos el agrupado de ambos dataframes**

In [ ]:
y=y.resample("3D",on="d_Date").mean()

In [ ]:
X=X.resample("3D",on="d_Date").agg(["min", "max", "mean", "sum", "std", "median",sumChange,minPctChange,maxChange])

**Renombramos columnas en X**

In [ ]:
X.shape

In [ ]:
X.columns = [ name+"|"+func for name, func in X.columns]
X.head()

**Desfase de y**

In [ ]:
y=y.shift(-2)#Desfase de dos periodos de tres días posteriores

**Unimos X y y**

In [ ]:
df_3_days=pd.concat([X,y],axis=1)

**Eliminamos nulos en la target**

In [ ]:
df_3_days.isnull().sum()

In [ ]:
df_3_days=df_3_days.dropna(subset=["tgt"])

**El resto de nulos en pasos posteriores son propensos a imputación**